# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys
import json

# Used to navigate to project root for api_keys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load saved city weather data for analysis
city_weather_data_csv = '../WeatherPy/output_data/cities.csv'

# Read and display the CSV with Pandas
city_weather_data = pd.read_csv(city_weather_data_csv)
city_weather_data.head()

,city_name,city_id,lat,lon,unix_dt,temp_max_c,temp_max_f,humidity,cloudiness,windspeed
0,Punta Arenas,3874787,-53.15,-70.92,1579926794,9.00,48.200,87,20,5.70
1,Massaguet,2428231,12.48,15.44,1579926800,14.00,57.200,19,0,4.60
2,Kavieng,2094342,-2.57,150.80,1579926800,28.95,84.110,73,42,7.11
3,Lebu,3883457,-37.62,-73.65,1579926800,12.87,55.166,70,0,3.16
4,Bathsheba,3374083,13.22,-59.52,1579926800,26.00,78.800,78,20,4.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create a heat map that displays the humidity for every city from the part I of the homework.
#01-Lesson-Plans/06-Python-APIs/3/Activities/07-Stu_Airport_Map/Solved/airport_heatmap.ipynb
# Store latitude and longitude in locations
locations = city_weather_data[["lat", "lon"]]

# Fill NaN values and convert to float
humidity = city_weather_data["humidity"].astype(float)


In [36]:
# Plot Heatmap
fig = gmaps.figure(zoom_level=2,center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=0,
                                 point_radius=4)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
ideal_weather_condition = city_weather_data[
    
    # A max temperature lower than 80 degrees but higher than 70
    (city_weather_data['temp_max_f'] > 70) & (city_weather_data['temp_max_f'] < 80) &
    
    # Wind speed less than 10 mph
    (city_weather_data['windspeed'] < 10) &
    
    # Zero cloudiness
    (city_weather_data['cloudiness'] == 0)
]
ideal_weather_condition

,city_name,city_id,lat,lon,unix_dt,temp_max_c,temp_max_f,humidity,cloudiness,windspeed
29,Venado Tuerto,3833062,-33.75,-61.97,1579926804,23.89,75.002,80,0,1.93
300,Tura,1254046,25.52,90.22,1579926718,22.63,72.734,36,0,1.84
344,Ampanihy,1078553,-24.70,44.75,1579926852,26.11,78.998,53,0,6.15
397,Kalia,1185272,23.04,89.63,1579926860,23.11,73.598,26,0,3.44
455,Mathbaria,1185239,22.29,89.96,1579926869,23.74,74.732,24,0,3.74
565,Pandamatenga,933178,-18.53,25.63,1579926884,22.49,72.482,68,0,4.17
566,General Roca,3855065,-39.03,-67.58,1579926885,26.00,78.800,27,0,4.60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = pd.DataFrame({
    'Hotel Name':'',
    'City':'',
    'Country':'',
    "Lat":ideal_weather_condition['lat'],
    "Lng":ideal_weather_condition['lon'],
    "orig_city_name":ideal_weather_condition['city_name']
})
# reset hotel_df index
hotel_df.reset_index(drop=True, inplace=True)

# create list to hold city coordinates for gmaps call
target_coordinates = []
# store ideal city location coordinates to target_coordinates list
for coordinates in range(0,len(hotel_df)):
    target_coordinates.append(f"{hotel_df['Lat'][coordinates]},{hotel_df['Lng'][coordinates]}")

In [11]:
# Ignore 'SettingWithCopyWarning' when saving values to hotel_df...copying from ideal_hotel_loc_df in function
pd.options.mode.chained_assignment = None  # default='warn'

def getFirstIdealHotelLoc(target_coordinates, ideal_hotel_loc_df, ideal_hotel_index):
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # initialize parms
    target_radius = 5000
    target_type = "lodging"
    fields = "address_component"
    
    # create parms list
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "fields": fields,
        "key": g_key
    }
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # convert response to json
    places_data = response.json()
    
    # initialize to '0' to capture first hotel returned in dataset
    index = 0
    
    # Isolate leading text in compound code (full address)
    # help parsing the full address to return only the city
    leadingText = places_data["results"][index]['plus_code']['compound_code'].split()[0]
    
    # extract hotel name, city, and country
    try:
        lodging = places_data["results"][index]['name']
    except:
        lodging = 'Hotel Name Error'
        pass
    
    try:
        city = places_data["results"][index]['plus_code']['compound_code'].split(leadingText)[1].split(',')[0]
    except:
        city = 'City Name Error'
        pass
    
    try:
        country = places_data["results"][index]['plus_code']['compound_code'].split()[-1]
    except:
        country = 'Country Name Error'
        pass
    
    ### 2. Format data for display in maps
    ideal_hotel_loc_df['Hotel Name'][ideal_hotel_index] = lodging
    ideal_hotel_loc_df['City'][ideal_hotel_index] = city
    ideal_hotel_loc_df['Country'][ideal_hotel_index] = country

In [12]:
### 1. Wrap above places extract into loop & save first returned value
for ideal_location in range(0,len(hotel_df)):
    getFirstIdealHotelLoc(target_coordinates[ideal_location], hotel_df, ideal_location)

In [13]:
# Check updated hotel_df.  All fields should be populated
hotel_df

,Hotel Name,City,Country,Lat,Lng,orig_city_name
0,La Bianca.,Venado Tuerto,Argentina,-33.75,-61.97,Venado Tuerto
1,Hotel RIKMAN Continental,Tura,India,25.52,90.22,Tura
2,Angora Hotel,Ampanihy,Madagascar,-24.70,44.75,Ampanihy
3,সামিয়া এন্টারপ্রাইজ,Kalia,Bangladesh,23.04,89.63,Kalia
4,বাহাদুরের চায়ের ষ্টোর,Mathbaria,Bangladesh,22.29,89.96,Mathbaria
5,Wildtrack Safaris Eco Lodge,Pandamatenga,Botswana,-18.53,25.63,Pandamatenga
6,Hotel El Recreo,General Roca,Argentina,-39.03,-67.58,General Roca


In [37]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [56]:
narrowed_city_df = ideal_weather_condition

In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))